In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import os

# Download latest version
path = kagglehub.dataset_download("ahmedmohamed2003/retail-store-sales-dirty-for-data-cleaning")
file_path = os.path.join(path,"retail_store_sales.csv")
df_raw = pd.read_csv(file_path)

print(df_raw.head())
print(df_raw.info())
print(df_raw.isna().sum())

C:\Users\Francisco\.cache\kagglehub\datasets\ahmedmohamed2003\retail-store-sales-dirty-for-data-cleaning\versions\1
  Transaction ID Customer ID       Category          Item  Price Per Unit  \
0    TXN_6867343     CUST_09     Patisserie   Item_10_PAT            18.5   
1    TXN_3731986     CUST_22  Milk Products  Item_17_MILK            29.0   
2    TXN_9303719     CUST_02       Butchers   Item_12_BUT            21.5   
3    TXN_9458126     CUST_06      Beverages   Item_16_BEV            27.5   
4    TXN_4575373     CUST_05           Food   Item_6_FOOD            12.5   

   Quantity  Total Spent  Payment Method Location Transaction Date  \
0      10.0        185.0  Digital Wallet   Online       2024-04-08   
1       9.0        261.0  Digital Wallet   Online       2023-07-23   
2       2.0         43.0     Credit Card   Online       2022-10-05   
3       9.0        247.5     Credit Card   Online       2022-05-07   
4       7.0         87.5  Digital Wallet   Online       2022-10-02   



In [161]:
#Quantity dtype to int
#Check transaction date dtype
#Check discount applied if boolean
#Clean data: calculate percentage of item, price per unit, quantity, total spent, and discount applied to drop or impute

#Updating datatypes
df_raw['Quantity'] = df_raw['Quantity'].astype('Int64')
df_raw['Transaction Date'] = pd.to_datetime(df_raw['Transaction Date'])

#Checking NaN Ratio: if NaNs differ significantly, it may be better to split dataset between locations.
nan_ratio_location = df_raw.groupby("Location").apply(lambda x: x.isna().sum() / df_raw.isna().sum())
print(nan_ratio_location.T, "\n")

#Checking Pct Missing (drop or not)
pct_missing = df_raw.apply(
                 lambda x: x.isna().sum() / len(x)
)
print(pct_missing)

#Item column is important, so we'll impute with Unknown_Item
df_raw['Item'] = df_raw['Item'].fillna("Unknown_Item")

#Checking if transactions with more than 1 item exist
print("\n", df_raw.groupby("Transaction ID")['Item'].count() is True)
#Items aren't duplicated
print(df_raw["Item"].duplicated().nunique() > len(df_raw["Item"]))

Location          In-store    Online
Transaction ID         NaN       NaN
Customer ID            NaN       NaN
Category               NaN       NaN
Item              0.497939  0.502061
Price Per Unit    0.469622  0.530378
Quantity          0.526490  0.473510
Total Spent       0.526490  0.473510
Payment Method         NaN       NaN
Location               NaN       NaN
Transaction Date       NaN       NaN
Discount Applied  0.499166  0.500834 

Transaction ID      0.000000
Customer ID         0.000000
Category            0.000000
Item                0.096461
Price Per Unit      0.048429
Quantity            0.048032
Total Spent         0.048032
Payment Method      0.000000
Location            0.000000
Transaction Date    0.000000
Discount Applied    0.333917
dtype: float64

 False
False


C:\Users\Francisco\AppData\Local\Temp\ipykernel_29236\4205404454.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_ratio_location = df_raw.groupby("Location").apply(lambda x: x.isna().sum() / df_raw.isna().sum())


In [162]:
#Fill Price Per Unit Randomly Grouped by Category
df_raw['Price Per Unit'] = df_raw.groupby("Category")['Price Per Unit'].transform(
                           lambda x: x.fillna(pd.Series(np.random.choice(x.dropna().to_numpy(), 
                                            size=int(x.isna().sum()), 
                                            replace=True),
                                            index=x.index[x.isna()]
    ))
)

#Fill Quantity Randomly Grouped by Category
df_raw['Quantity'] = df_raw.groupby("Category")['Quantity'].transform(
                           lambda x: x.fillna(pd.Series(np.random.choice(x.dropna().to_numpy(), 
                                            size=int(x.isna().sum()), 
                                            replace=True),
                                            index=x.index[x.isna()]
    ))
)

In [163]:
#Estimated Total Spent
df_raw['Estimated Total Spent'] = df_raw['Price Per Unit'] * df_raw['Quantity'].round(2)
#Can I use total difference to infer price? What if discount is baked into price? if False, then I can
df_raw.loc[df_raw['Discount Applied'] == True, 'Total Spent'].equals(
    df_raw.loc[df_raw['Discount Applied'] == True, 'Estimated Total Spent']
)

False

In [164]:
#Filling NA on Discount
discount_mask = df_raw['Discount Applied'].isna() & df_raw['Total Spent'].notna()
df_raw.loc[discount_mask,'Discount Applied'] = (df_raw.loc[discount_mask, 'Total Spent'] < df_raw.loc[discount_mask, 'Estimated Total Spent'])

#Fill Total Spent
total_mask = df_raw['Total Spent'].isna()
df_raw.loc[total_mask,'Total Spent'] = (df_raw.loc[total_mask, 'Price Per Unit'] * df_raw.loc[total_mask, 'Quantity']).round(2)

In [171]:
#Check if drop < 5%
null_discount = df_raw['Discount Applied'].isna()
print("Drop share overall:", null_discount.mean())

#Check if ratio by category or location is close to avoid bias
print(null_discount.groupby(df_raw['Location']).mean().sort_values())
print(null_discount.groupby(df_raw['Category']).mean().sort_values())

#Drop NaNs
n_drop = df_raw['Discount Applied'].isna().sum()
df_clean = df_raw.dropna(subset=['Discount Applied']).copy()
print(f"Dropped {n_drop} rows ({n_drop/len(df_raw):.2%})")

print(df_clean.isna().sum())
df_clean.to_csv("cleaned_retail_store_sales.csv")


Drop share overall: 0.016779324055666005
Location
Online      0.014951
In-store    0.018647
Name: Discount Applied, dtype: float64
Category
Milk Products                         0.011995
Beverages                             0.014678
Butchers                              0.015306
Computers and electric accessories    0.015404
Electric household essentials         0.015713
Furniture                             0.017599
Food                                  0.020781
Patisserie                            0.022906
Name: Discount Applied, dtype: float64
Dropped 211 rows (1.68%)
Transaction ID           0
Customer ID              0
Category                 0
Item                     0
Price Per Unit           0
Quantity                 0
Total Spent              0
Payment Method           0
Location                 0
Transaction Date         0
Discount Applied         0
Estimated Total Spent    0
dtype: int64
